In [1]:
import os
import pynq
import numpy as np
# os.environ['EMCONFIG_PATH'] = os.environ['PWD']
ol=pynq.Overlay("krnl_matmulbertl.xclbin")


/data/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [590]:
from pynq import allocate
Nbanks=8
Nmat=3
Tsize=1024
Nvec=14

In [591]:
low = 1 # -2^15
high =  200# 2^15-1
source_v_np = np.random.randint(low, high, dtype=np.int16, size=(Tsize,Nvec))
source_v_np.shape

(1024, 14)

In [592]:
low = -1000
high = 1000
source_w_np = np.random.randint(low, high, dtype=np.int16, size=(Nmat*Tsize,Tsize))
source_w_np[100,13] = 2
source_w_np.shape


(3072, 1024)

In [593]:
source_w_split_np = []
for i in range(Nbanks):
    source_w_split_np.append(np.zeros((Nmat*Tsize,Tsize//Nbanks)))

In [594]:
source_w_split_np[0].shape

(3072, 128)

In [595]:
# save the split matrices of the reference values for the loading
source_w_split_np = np.hsplit(source_w_np, Nbanks)
source_w_split_np[0].shape

(3072, 128)

In [596]:
outbuf_np = np.zeros((Nvec, Tsize), dtype=np.int32)
outbuf_np.shape

(14, 1024)

In [597]:
outbuf_np = np.matmul(source_w_np, source_v_np, dtype=np.int32)
outbuf_np, outbuf_np.shape

(array([[  975209,  1956978,  -947947, ...,   268320,  -215044, -1190427],
        [ 1056084,   170725,    55925, ...,   919461,  -474195,   618715],
        [ 5726443,  4397617,  3614665, ...,  2452041,  4673051,  5300048],
        ...,
        [ -672202,    19015,  1456368, ...,  2017827,  -978472,  2537446],
        [  263658,  1065093,  2462972, ...,  -301257,    47755,  4091734],
        [-1843463, -3288543, -2834452, ..., -2284886, -2607814, -2938570]],
       dtype=int32),
 (3072, 14))

In [598]:
source_w = [
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int16, target=ol.HBM0),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int16, target=ol.HBM4),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int16, target=ol.HBM8),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int16, target=ol.HBM12),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int16, target=ol.HBM16),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int16, target=ol.HBM20),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int16, target=ol.HBM24),
  pynq.allocate(shape=(Nmat*Tsize,Tsize//Nbanks), dtype=np.int16, target=ol.HBM26)]
 

In [599]:
source_w[0].shape, source_w_split_np[0].shape

((3072, 128), (3072, 128))

In [600]:
for i in range(len(source_w)):
#     source_w[i][:] = np.random.randint(-2^15, high=2^15-1, dtype=np.int16, size=(Nmat*Tsize//Nbanks,Tsize))
    source_w[i][:] = source_w_split_np[i]

In [601]:
source_w[0].shape


(3072, 128)

In [602]:
for i in range(8):
    source_w[i].sync_to_device()

In [603]:
source_v = pynq.allocate(shape=(Nvec,Tsize), dtype=np.int16, target=ol.HBM14)
print("source_v shape", source_v.shape)
# source_v[:] = np.random.randint(-2^15, high=2^15-1, dtype=np.int16, size=(Tsize,Nvec))
source_v[:] = source_v_np.T
print(type(source_v))
source_v.sync_to_device()
outbuf = pynq.allocate((Tsize*Nmat,Nvec), dtype=np.int32, target=ol.HBM14)
print("outbuf shape", outbuf.shape)

source_v shape (14, 1024)
<class 'pynq.buffer.PynqBuffer'>
outbuf shape (3072, 14)


In [604]:
ol.feeder_1.call(
	source_v,
	source_w[0],
	source_w[1],
	source_w[2],
	source_w[3],
	source_w[4],
	source_w[5],
	source_w[6],
	source_w[7],
    outbuf, 0)

outbuf.sync_from_device()

In [605]:
outbuf.shape

(3072, 14)

In [606]:
outbuf_np.shape

(3072, 14)

In [607]:
source_v

PynqBuffer([[125, 153, 155, ..., 152, 110,  27],
            [ 72, 116, 143, ..., 129, 109,   7],
            [197, 182, 162, ...,   9, 170, 174],
            ...,
            [ 67,  27,  15, ...,  14,  69, 120],
            [ 84,  11, 100, ..., 106, 166,  15],
            [ 11, 103, 171, ...,  43,  47,   4]], dtype=int16)

In [608]:
source_w[0]

PynqBuffer([[-549, -792, -813, ..., -353, -608,  610],
            [-758, -791, -698, ...,  241, -126,  -36],
            [-486, -451, -845, ..., -792, -672,   61],
            ...,
            [-135, -301, -234, ...,  115,  987, -240],
            [-274,  780,  499, ..., -372,  864, -939],
            [-325,  165, -909, ...,  850, -251, -804]], dtype=int16)

In [609]:
outbuf

PynqBuffer([[  975209,  1956978,  -947947, ...,   268320,  -215044,
             -1190427],
            [ 1056084,   170725,    55925, ...,   919461,  -474195,
               618715],
            [ 5726443,  4397617,  3614665, ...,  2452041,  4673051,
              5300048],
            ...,
            [ -672202,    19015,  1456368, ...,  2017827,  -978472,
              2537446],
            [  263658,  1065093,  2462972, ...,  -301257,    47755,
              4091734],
            [-1843463, -3288543, -2834452, ..., -2284886, -2607814,
             -2938570]], dtype=int32)

In [610]:
outbuf_np

array([[  975209,  1956978,  -947947, ...,   268320,  -215044, -1190427],
       [ 1056084,   170725,    55925, ...,   919461,  -474195,   618715],
       [ 5726443,  4397617,  3614665, ...,  2452041,  4673051,  5300048],
       ...,
       [ -672202,    19015,  1456368, ...,  2017827,  -978472,  2537446],
       [  263658,  1065093,  2462972, ...,  -301257,    47755,  4091734],
       [-1843463, -3288543, -2834452, ..., -2284886, -2607814, -2938570]],
      dtype=int32)

In [611]:
outbuf.any() == outbuf_np.any()

PynqBuffer(True)